In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np


print("Loading the dataset...")
try:
    df = pd.read_excel('Final.xlsx', sheet_name='Sheet1', engine='openpyxl')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: The file 'Final.xlsx' was not found.")
    print("Please make sure the file is uploaded and correctly named.")
    exit()
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")
    exit()

# --- 2. Data Cleaning and Preprocessing ---
print("\nCleaning and preprocessing the data...")


def clean_market_value(value):
    if isinstance(value, str):
        value = value.replace('€', '').replace('m', '')
        
        if value:
            return float(value)
    return value

df['Market value'] = df['Market value'].apply(clean_market_value)

threshold = 0.5 * len(df)
df = df.dropna(axis=1, thresh=threshold)
print(f"Dropped columns with more than 50% missing data. Remaining columns: {df.shape[1]}")


numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
for col in numerical_cols:
    df[col].fillna(df[col].mean(), inplace=True)


features_to_drop = ['Unnamed: 0', 'Unnamed: 0.1', 'Player', 'Club_x', 'PK_x', 'PKatt_x']

features = [col for col in df.columns if col not in ['Market value'] + features_to_drop]


X = df[features]
y = df['Market value']

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=np.number).columns.tolist()

print(f"Selected numerical features: {numerical_features}")
print(f"Selected categorical features: {categorical_features}")


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])


print("\nSplitting data into training and testing sets (80/20)...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")


print("\nCreating and training the Random Forest Regressor model...")
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

# Fit the model to the training data.
model_pipeline.fit(X_train, y_train)
print("Model training complete.")

# --- 5. Make Predictions and Evaluate the Model ---
print("\nMaking predictions on the test set...")
y_pred = model_pipeline.predict(X_test)

# Evaluate the model's performance.
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Evaluation Results:")
print(f"Mean Absolute Error (MAE): €{mae:.2f}m")
print(f"R-squared (R²): {r2:.4f}")


print("\n--- Example Prediction on a New Player ---")
new_player_data = {
    'Player': ['New Player'],
    'Nation': ['BRA'],
    'Pos': ['FW'],
    'Leauge': ['Ligue1'],
    'Age': [22],
    'MP': [35],
    'Starts': [30],
    'Min': [3000],
    'Gls': [25],
    'Ast': [10],
    'PK': [2],
    'PKatt': [3],
    'CrdY': [3],
    'CrdR': [0],
    'Gls90': [0.75],
    'Ast90': [0.3],
    'G+A': [1.05],
    'Gls+Ast': [35],
    'Sh': [100],
    'SoT': [60],
    'FK': [1],
    'SoT%': [60],
    'Sh/90': [3],
    'SoT/90': [1.8],
    'G/Sh': [0.25],
    'G/SoT': [0.4],
    'Tackle': [10],
    'TackleW': [5],
    'TakleD': [5],
    'Tkl%': [50],
    'Press': [200],
    'Succ_x': [50],
    '%': [25],
    'Blocks': [5],
    'ShotB': [3],
    'PassB': [2],
    'Int': [5],
    'Clr': [5],
    'Passes Completed': [1000],
    'Passes Attempted': [1200],
    'Cmp%': [83.3],
    'Touches': [1800],
    'Succ_y': [100],
    'Att': [200],
    'Succ%': [50],
    '#Pl': [10]
}

new_player_df = pd.DataFrame(new_player_data)

new_player_df = new_player_df[features]

predicted_value = model_pipeline.predict(new_player_df)
print(f"Predicted market value for the new player: €{predicted_value[0]:.2f}m")


Loading the dataset...
Dataset loaded successfully.

Cleaning and preprocessing the data...
Dropped columns with more than 50% missing data. Remaining columns: 52
Selected numerical features: ['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'CrdY', 'CrdR', 'Gls90', 'Ast90', 'G+A', 'Gls+Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'FK', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Tackle', 'TackleW', 'TakleD', 'Tkl%', 'Press', 'Succ_x', '%', 'Blocks', 'ShotB', 'PassB', 'Int', 'Clr', 'Passes Completed', 'Passes Attempted', 'Cmp%', 'Touches', 'Succ_y', 'Att', 'Succ%', '#Pl']
Selected categorical features: ['Nation', 'Pos', 'Leauge']

Splitting data into training and testing sets (80/20)...
Training set size: 413 samples
Testing set size: 104 samples

Creating and training the Random Forest Regressor model...
Model training complete.

Making predictions on the test set...
Evaluation Results:
Mean Absolute Error (MAE): €13.42m
R-squared (R²): 0.3351

--- Example Prediction on a New Player ---
Predicted ma

In [ ]:
import joblib


joblib.dump(model_pipeline, "model/market_value_model.pkl")
joblib.dump(features, "model/features.pkl")


['model/features.pkl']

In [3]:
import pandas as pd
df = pd.read_excel('Final.xlsx', sheet_name='Sheet1', engine='openpyxl')
print("Dataset loaded successfully.")

Dataset loaded successfully.


In [4]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Player,Market value,Nation,Pos,Club_x,Leauge,Age,MP,...,Int,Clr,Passes Completed,Passes Attempted,Cmp%,Touches,Succ_y,Att,Succ%,#Pl
0,0,0,Kylian Mbappé,€180.00m,FRA,FW,Paris S-G,Ligue1,20.0,49.0,...,7.0,3.0,1152.0,1494.0,77.11,2120.0,132.0,247.0,53.44,152.0
1,1,1,Raheem Sterling,€128.00m,ENG,FW,Manchester City,Premier Leauge,24.0,60.0,...,27.0,13.0,1963.0,2414.0,81.32,3209.0,139.0,258.0,53.88,151.0
2,2,2,Neymar,€128.00m,BRA,"MF,FW",Paris S-G,Ligue1,27.0,32.0,...,10.0,2.0,1588.0,2016.0,78.77,2696.0,164.0,292.0,56.16,184.0
3,3,3,Sadio Mané,€120.00m,SEN,FW,Liverpool,Premier Leauge,27.0,64.0,...,28.0,11.0,1728.0,2214.0,78.05,3050.0,128.0,212.0,60.38,142.0
4,4,4,Mohamed Salah,€120.00m,EGY,FW,Liverpool,Premier Leauge,27.0,65.0,...,12.0,5.0,1516.0,2029.0,74.72,2893.0,137.0,229.0,59.83,148.0


In [5]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Player', 'Market value', 'Nation', 'Pos',
       'Club_x', 'Leauge', 'Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'PK_x',
       'PKatt_x', 'CrdY', 'CrdR', 'Gls90', 'Ast90', 'G+A', 'Gls+Ast', 'PK',
       'PKatt', 'Sh', 'SoT', 'FK', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT',
       'Tackle', 'TackleW', 'TakleD', 'Tkl%', 'Press', 'Succ_x', '%', 'Blocks',
       'ShotB', 'PassB', 'Int', 'Clr', 'Passes Completed', 'Passes Attempted',
       'Cmp%', 'Touches', 'Succ_y', 'Att', 'Succ%', '#Pl'],
      dtype='object')

In [5]:
df['Pos']

0         FW
1         FW
2      MF,FW
3         FW
4         FW
       ...  
512    MF,FW
513    MF,FW
514    MF,FW
515    MF,FW
516       FW
Name: Pos, Length: 517, dtype: object

In [7]:
df['Leauge'].unique()

array(['Ligue1', 'Premier Leauge', 'Bundesliga', 'La Liga', nan,
       'Serie A'], dtype=object)

In [8]:
df['Pos'].unique()

array(['FW', 'MF,FW', 'MF', 'DF', nan, 'MF,DF', 'WB', 'DF,MF'],
      dtype=object)